In [274]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [275]:
data = loadmat('ex4data1.mat')
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [276]:
X = data['X']
y = data['y']
y_ori = y # for prediction use
X.shape, y.shape#看下维度

((5000, 400), (5000, 1))

In [277]:
def expand_y(y):
    res = []
    for i in y:
        temp = np.zeros(10)
        temp[i - 1] = 1
        res.append(temp)
    return np.matrix(res)

In [278]:
y = expand_y(y)
y.shape

(5000, 10)

In [241]:
def serialize(a, b):
    return np.concatenate((np.ravel(a), np.ravel(b)))

In [242]:
def deserialize(seq):
#     """into ndarray of (25, 401), (10, 26)"""
    return seq[:25 * 401].reshape(25, 401), seq[25 * 401:].reshape(10, 26)

In [254]:
weights = loadmat("ex4weights.mat")
theta1 = weights["Theta1"]
theta2 = weights["Theta2"]
theta = serialize(theta1, theta2) # 扁平化参数，25*401+10*26=10285
theta1.shape, theta2.shape, theta.shape

((25, 401), (10, 26), (10285,))

In [255]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [256]:
def forward(X, theta1, theta2):
    X = np.matrix(X)
    theta1 = np.matrix(theta1)
    theta2 = np.matrix(theta2)
    a1 = np.insert(X, 0, 1, axis=1)
    z2 = a1 * theta1.T
    a2 = sigmoid(z2)
    a2 = np.insert(a2, 0, 1, axis=1)
    z3 = a2 * theta2.T
    h = sigmoid(z3)
    return a1, z2, a2, z3, h

In [257]:
def reg_cost(theta, X, y, lamda):
    m = X.shape[0]
    theta1, theta2 = deserialize(theta)
    a1, z2, a2, z3, h = forward(X, theta1, theta2)
    first = np.multiply(-y, np.log(h))
    second = np.multiply((1 - y), np.log(1 - h))
    sums = np.sum(first - second) / m 
    
    extra = (np.sum(np.power(theta1[:, 1:], 2)) + np.sum(np.power(theta2[:, 1:], 2))) * (lamda / (2 * m))
    return sums + extra

In [258]:
lamda = 1
cost = reg_cost(theta, X, y, lamda)
cost

0.38376985909092365

In [259]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [260]:
def back_propagate(theta, X, y, lamda):
    m = X.shape[0]
    theta1, theta2 = deserialize(theta)
    a1, z2, a2, z3, h = forward(X, theta1, theta2)
     # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)
    
    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)
        
        d3t = ht - yt  # (1, 10)
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * lamda) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * lamda) / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    return grad

In [261]:
grad = back_propagate(theta, X, y, lamda)
grad.shape

(10285,)

In [263]:
from scipy.optimize import minimize
init_theta = random_init(10285)  # 25*401 + 10*26
# minimize the objective function
fmin = minimize(fun=reg_cost, x0=theta, args=(X, y, lamda), 
                method='TNC', jac=back_propagate, options={'maxiter': 250})
fmin

     fun: 0.30962438181592294
     jac: array([-3.35708233e-05,  1.64790301e-13, -3.42321429e-14, ...,
       -2.81646246e-05, -2.68355362e-05, -1.08223043e-05])
 message: 'Max. number of function evaluations reached'
    nfev: 250
     nit: 10
  status: 3
 success: False
       x: array([-1.01104769e-01,  8.23951505e-10, -1.71160714e-10, ...,
        4.80344726e-01,  2.18933231e+00, -1.80157596e+00])

In [264]:
final_theta = fmin.x

In [265]:
final_theta.shape

(10285,)

In [266]:
theta1, theta2 = deserialize(final_theta)

In [270]:
a1, z2, a2, z3, h = forward(X, theta1, theta2)
y_pred = np.array(np.argmax(h, axis=1) + 1)
y_pred.shape, y.shape


((5000, 1), (5000, 10))

In [273]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y_ori)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ('accuracy = {0}%'.format(accuracy * 100))

accuracy = 99.58%
